# About

This script was created by Christian Thomas at SkyTruth to aggregate index and band values for last-mined polygons. It also supports this processing for custom polygons.

The code processes RMA harmonized, Surface Reflectance Median pixel composites. The code does not identify EPA Level 4 ecoregions (US_L4CODE) which is an attribute used in other processing scripts, this data is added locally.

https://code.earthengine.google.com/a92026e76f77c3c49244e22c81065304


# 1. Install, Authenticate, and Initialize Earth Engine

In [ ]:
!earthengine authenticate

In [ ]:
# Imports
import ee
from ee import batch

# Intialize EE
ee.Initialize()


# 2. Prepare Global Variables

In [ ]:
# Global Variables

# Specify the study area and define the export geometry
studyArea = ee.FeatureCollection('users/skytruth-data/Plos_MTM_Fusion_Table_Backup/plosScriptFusionTableBackup/studyArea')
exportGeometry = studyArea.geometry()

# Define EPA Level-4 Ecoregion Sample Sites (selected sites which did not experience mining activity and serve as a 
# baseline for undisturbed forest).
epaEcoregionTestSites = ee.FeatureCollection('users/skytruth-data/MTR/allEcoregionTestSites_update')

# Specify the Last Mined and Median Pixel Imagery
lastMinedRaw = ee.Image('users/skytruth-data/MTR/LastMined_Raw')
medianPixelComposite = ee.ImageCollection('users/skytruth-data/MTR/SR_RMA_Harmonized_medianComposite')

# Get the scale of a landsat pixel
lsScale = medianPixelComposite.first().projection().nominalScale()

# A list whose length is equal to the number of years of imagery to process.
yearSequence = ee.List.sequence(0,35)  # 1984-2019
# # yearSequence = ee.List.sequence(0,10)   # 1984-1994
# # yearSequence = ee.List.sequence(11,21)  # 1995-2005
# # yearSequence = ee.List.sequence(26,35)  # 2010-2019

# A list of all bands
bands = ['B','G','R','NIR','SWIR1','SWIR2','NDVI','NDMI','EVI','SAVI','MSAVI','NBR','NBR2']

# For processing of pre-filtered, or partially processed data, define the input on line 30, samples provided.
# NOTE : This data is derived from partially processed output of the lastMinedRaw data.
lastMinedCleaned = ee.FeatureCollection('users/skytruth-data/MTR/20201026_lastMined_before_2015_gte_10px')


# For processing of custom polygon data which has not been pre or partially processed and is not derived from 
# lastMinedRaw data, samples provided.
inputPolygons = ee.FeatureCollection('users/christian/2021-03-08_mtm_test_data')


In [ ]:
# print(inputPolygons.first().getInfo())
# print(lastMinedCleaned.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-83.18160048423242, 37.42912792992443], [-83.18154701091741, 37.428664149893194], [-83.18090491888775, 37.42837882690549], [-83.17916585034511, 37.42923048020517], [-83.17888048461919, 37.429868147502475], [-83.17900975790107, 37.43032299863417], [-83.17940215511369, 37.430728742654736], [-83.18002643711303, 37.43153582041168], [-83.18040550051, 37.43228941143527], [-83.18093165581459, 37.4322671621904], [-83.18143998239191, 37.43199514080157], [-83.18119477275334, 37.4313352157496], [-83.18105650975878, 37.4304968432079], [-83.18146230386363, 37.42958275481484], [-83.18160048423242, 37.42912792992443]]]}, 'id': '00000000000000000001', 'properties': {'Name': 'Untitled Polygon', 'altitudeMo': '', 'begin': '', 'descriptio': '', 'end': '', 'extrude': 0, 'icon': '', 'layer': 'laurel_fork', 'path': '/Users/christian/Desktop/mtm_arm_test_data/laurel_fork.shp|layername=laurel_fork', 'tessellate': 1, 'timestamp': '', 'visibi

# 3. Prepare Processing Functions

In [ ]:
"""
====================
Processing Functions
====================
"""

def clipping (image):
  clippedImg = image.clip(studyArea)
  return(clippedImg)

def cleanup (image):
  year = ee.Number(image.get('year'))
  pxArea = ee.Image.pixelArea().multiply(image).divide(year).rename('area_cal').toFloat()
  return(image.addBands(pxArea))

def pixelAreaGPC (image):
  scale = image.projection().nominalScale()
  scaleArea = scale.multiply(scale)
  final = image.addBands(scaleArea).rename('B','G','R','NIR','SWIR1','SWIR2','NDVI','NDMI','EVI','SAVI','MSAVI','NBR','NBR2','pxArea')
  return(final)

def pixelAreaLM (image):
  year = ee.Number(image.get('year'))
  scaleArea = lsScale.multiply(lsScale)
  final = image.addBands(scaleArea).rename('lastMined','area')
  return(final)

def vectorize (image):
  year = image.get('year')
  vector = image.reduceToVectors(
      reducer=ee.Reducer.sum(),
      geometry=exportGeometry,
      scale=30,
      crs='EPSG:5072',
      labelProperty='lastMined',
      maxPixels=1e10
  ).set('year',year)
  filteredVector = vector.filterMetadata('lastMined','not_equals',0)
  return(filteredVector)

def featID (feature):
  fid = feature.id()
  featureID = feature.set('ID',fid)
  return(featureID)

def areaCalcVector (feat):
  calculatedArea = feat.area().toFloat()
  fin = feat.set('sum',calculatedArea)
  return(fin)

# 4. Processing Data

In [ ]:
# 5 Year Bucket
lastMined_5yr_1984 = ee.ImageCollection([lastMinedRaw.eq(1984)]).max().multiply(1984).toInt().rename('lastMined').set('year',1984)
lastMined_5yr_1985 = ee.ImageCollection([lastMinedRaw.eq(1985),lastMinedRaw.eq(1986),lastMinedRaw.eq(1987),lastMinedRaw.eq(1988),lastMinedRaw.eq(1989)]).max().multiply(1985).toInt().rename('lastMined').set('year',1985)
lastMined_5yr_1990 = ee.ImageCollection([lastMinedRaw.eq(1990),lastMinedRaw.eq(1991),lastMinedRaw.eq(1992),lastMinedRaw.eq(1993),lastMinedRaw.eq(1994)]).max().multiply(1990).toInt().rename('lastMined').set('year',1990)
lastMined_5yr_1995 = ee.ImageCollection([lastMinedRaw.eq(1995),lastMinedRaw.eq(1996),lastMinedRaw.eq(1997),lastMinedRaw.eq(1998),lastMinedRaw.eq(1999)]).max().multiply(1995).toInt().rename('lastMined').set('year',1995)
lastMined_5yr_2000 = ee.ImageCollection([lastMinedRaw.eq(2000),lastMinedRaw.eq(2001),lastMinedRaw.eq(2002),lastMinedRaw.eq(2003),lastMinedRaw.eq(2004)]).max().multiply(2000).toInt().rename('lastMined').set('year',2000)
lastMined_5yr_2005 = ee.ImageCollection([lastMinedRaw.eq(2005),lastMinedRaw.eq(2006),lastMinedRaw.eq(2007),lastMinedRaw.eq(2008),lastMinedRaw.eq(2009)]).max().multiply(2005).toInt().rename('lastMined').set('year',2005)
lastMined_5yr_2010 = ee.ImageCollection([lastMinedRaw.eq(2010),lastMinedRaw.eq(2011),lastMinedRaw.eq(2012),lastMinedRaw.eq(2013),lastMinedRaw.eq(2014)]).max().multiply(2010).toInt().rename('lastMined').set('year',2010)
lastMined_5yr_2015 = ee.ImageCollection([lastMinedRaw.eq(2015)]).max().multiply(2015).toInt().rename('lastMined').set('year',2015)

# Create the Image Collections from the created bucketed images
lastMined_5yr_IC = ee.ImageCollection([lastMined_5yr_1984,lastMined_5yr_1985,lastMined_5yr_1990,lastMined_5yr_1995,lastMined_5yr_2000,lastMined_5yr_2005,lastMined_5yr_2010,lastMined_5yr_2015])

In [ ]:
# Map the functions to add area data to images as bands
medianPixelCompositeArea = medianPixelComposite.map(pixelAreaGPC)

# Map the functions to add area data to images as bands
lastMinedArea_5yr = lastMined_5yr_IC.map(pixelAreaLM)

# Vectorize the Last Mined Image Collections
vectorLastMined_5yr = lastMinedArea_5yr.map(vectorize).flatten()

# Generate an ID for each polygon in the vectorized last mine file
vectorLastMined_5yr_ID = vectorLastMined_5yr.map(featID)

# Generate an ID for each polygon in the EPA Ecoregion Site file
epaEcoregionTestSites_ID = epaEcoregionTestSites.map(featID)

# Add Area and ID to the custom inputPolygons
inputPolygons_area = inputPolygons.map(areaCalcVector)
inputPolygons_ID = inputPolygons_area.map(featID)

# Filter Input Polygons to only preserve the calculated ID and sum attribute, for easier processing later.
inputPolygons_ID_only =inputPolygons_ID.select('ID','sum')

In [ ]:
# print(inputPolygons_ID.first().getInfo())
print(inputPolygons_ID_only.first().getInfo())
print(vectorLastMined_5yr_ID.first().getInfo())
print(epaEcoregionTestSites_ID.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-83.18160048423242, 37.42912792992443], [-83.18154701091741, 37.428664149893194], [-83.18090491888775, 37.42837882690549], [-83.17916585034511, 37.42923048020517], [-83.17888048461919, 37.429868147502475], [-83.17900975790107, 37.43032299863417], [-83.17940215511369, 37.430728742654736], [-83.18002643711303, 37.43153582041168], [-83.18040550051, 37.43228941143527], [-83.18093165581459, 37.4322671621904], [-83.18143998239191, 37.43199514080157], [-83.18119477275334, 37.4313352157496], [-83.18105650975878, 37.4304968432079], [-83.18146230386363, 37.42958275481484], [-83.18160048423242, 37.42912792992443]]]}, 'id': '00000000000000000001', 'properties': {'ID': '00000000000000000001', 'sum': 64217.524958958806}}
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-84.6501080003404, 36.67982354001174], [-84.65014837168746, 36.67955766525272], [-84.65048479438394, 36.67958957014939], [-8

# 5. Iterate through the vectorized Last Mined collections

In [ ]:
"""
Function to Process every polygon in a vector layer, generating mean statistics
for all indices, for each year in the specified yearSequence range.

The featureProcessing function iterates through the yearSequence list, for each
feature in the feature collection specified by polygonProcessing. For each feature
the function is applied for every year in the year sequence list.

featureProcessing takes 2 arguments, year and feat. Year is provided by yearSequence,
feat comes from the feature in a feature collection being processed. For the first
iteration of the code, feat comes from the first feature in a featureCollection 
specified by the polygonProcessing function. Then it gets the next feature from that
collection the next time it is iterated.
"""

def polygonProcessing (feature):
  
  # Feature Processing
  def featureProcessing (year, feat):
    year = ee.Number(year).toInt()
    feat = ee.Feature(feat)
    
    realYearNumeric = ee.Number(1984).toInt().add(year)
    realYearString = str(realYearNumeric)
    
    annualImage = medianPixelCompositeArea.filterMetadata('year','equals',realYearNumeric)
    
    reduction = annualImage.first().select(bands).reduceRegion(
        geometry=feature.geometry(),
        reducer=ee.Reducer.mean(),
        scale=30,
        maxPixels=1e13)
    
    b_mean = ee.Number(reduction.get('B'))
    g_mean = ee.Number(reduction.get('G'))
    r_mean = ee.Number(reduction.get('R'))
    nir_mean = ee.Number(reduction.get('NIR'))
    swir1_mean = ee.Number(reduction.get('SWIR1'))
    swir2_mean = ee.Number(reduction.get('SWIR2'))
    ndvi_mean = ee.Number(reduction.get('NDVI'))
    ndmi_mean = ee.Number(reduction.get('NDMI'))
    evi_mean = ee.Number(reduction.get('EVI'))
    savi_mean = ee.Number(reduction.get('SAVI'))
    msavi_mean = ee.Number(reduction.get('MSAVI'))
    nbr_mean = ee.Number(reduction.get('NBR'))
    nbr2_mean = ee.Number(reduction.get('NBR2'))

    b_mean_field_pt1 = ee.String('B_').cat(ee.Number(realYearNumeric).int().format())
    g_mean_field_pt1 = ee.String('G_').cat(ee.Number(realYearNumeric).int().format())
    r_mean_field_pt1 = ee.String('R_').cat(ee.Number(realYearNumeric).int().format())
    nir_mean_field_pt1 = ee.String('NIR_').cat(ee.Number(realYearNumeric).int().format())
    swir1_mean_field_pt1 = ee.String('SWIR1_').cat(ee.Number(realYearNumeric).int().format())
    swir2_mean_field_pt1 = ee.String('SWIR2_').cat(ee.Number(realYearNumeric).int().format())
    ndvi_mean_field_pt1 = ee.String('NDVI_').cat(ee.Number(realYearNumeric).int().format())
    ndmi_mean_field_pt1 = ee.String('NDMI_').cat(ee.Number(realYearNumeric).int().format())
    evi_mean_field_pt1 = ee.String('EVI_').cat(ee.Number(realYearNumeric).int().format())
    savi_mean_field_pt1 = ee.String('SAVI_').cat(ee.Number(realYearNumeric).int().format())
    msavi_mean_field_pt1 = ee.String('MSAVI_').cat(ee.Number(realYearNumeric).int().format())
    nbr_mean_field_pt1 = ee.String('NBR_').cat(ee.Number(realYearNumeric).int().format())
    nbr2_mean_field_pt1 = ee.String('NBR2_').cat(ee.Number(realYearNumeric).int().format())

    b_mean_field_pt2 = ee.String(b_mean_field_pt1)
    g_mean_field_pt2 = ee.String(g_mean_field_pt1)
    r_mean_field_pt2 = ee.String(r_mean_field_pt1)
    nir_mean_field_pt2 = ee.String(nir_mean_field_pt1)
    swir1_mean_field_pt2 = ee.String(swir1_mean_field_pt1)
    swir2_mean_field_pt2 = ee.String(swir2_mean_field_pt1)
    ndvi_mean_field_pt2 = ee.String(ndvi_mean_field_pt1)
    ndmi_mean_field_pt2 = ee.String(ndmi_mean_field_pt1)
    evi_mean_field_pt2 = ee.String(evi_mean_field_pt1)
    savi_mean_field_pt2 = ee.String(savi_mean_field_pt1)
    msavi_mean_field_pt2 = ee.String(msavi_mean_field_pt1)
    nbr_mean_field_pt2 = ee.String(nbr_mean_field_pt1)
    nbr2_mean_field_pt2 = ee.String(nbr2_mean_field_pt1)
    
    return(feat.set(
        b_mean_field_pt2, b_mean,
        g_mean_field_pt2, g_mean,
        r_mean_field_pt2, r_mean,
        nir_mean_field_pt2, nir_mean,
        swir1_mean_field_pt2, swir1_mean,
        swir2_mean_field_pt2, swir2_mean,
        ndvi_mean_field_pt2, ndvi_mean,
        ndmi_mean_field_pt2, ndmi_mean,
        evi_mean_field_pt2, evi_mean,
        savi_mean_field_pt2, savi_mean,
        msavi_mean_field_pt2, msavi_mean,
        nbr_mean_field_pt2, nbr_mean,
        nbr2_mean_field_pt2, nbr2_mean
    ))

  finalFeature = ee.Feature(yearSequence.iterate(featureProcessing,feature))
  return(finalFeature)

In [ ]:
# Map polygonProcessing to vectorized Last Mined collections
processedLastMined_5yr = lastMinedCleaned.map(polygonProcessing)

# Map polygonProcessing to EPA Ecoregion Sites
fullyProcessed_EPA_ER_Sites = epaEcoregionTestSites_ID.map(polygonProcessing)

# Map polygonProcessing to custom input Polygons
fullyProcessedCustomPolygons = inputPolygons_ID_only.map(polygonProcessing)

# 6. Inspect Size and First Feature of Processed Datasets

In [ ]:
# print('Last Mined Polygon Data')
# print('Size: '+str(processedLastMined_5yr.size().getInfo()))
# print('First: '+str(processedLastMined_5yr.first().getInfo()))

print('EPA Ecoregion Sample Site Data')
print('Size: '+str(fullyProcessed_EPA_ER_Sites.size().getInfo()))
print('First: '+str(fullyProcessed_EPA_ER_Sites.first().getInfo()))

print('Custom Polygon Data')
print('Size: '+str(fullyProcessedCustomPolygons.size().getInfo()))
print('First: '+str(fullyProcessedCustomPolygons.first().getInfo()))

EPA Ecoregion Sample Site Data
Size: 351
First: {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-79.90932931898799, 38.34794356538606], [-79.90667170050511, 38.34094275911733], [-79.89933201909204, 38.34461255746751], [-79.90040669673772, 38.349392732542995], [-79.90207886592893, 38.354538526370675], [-79.90804514797014, 38.35797206231018], [-79.90932931898799, 38.34794356538606]]]}, 'id': '00000000000000000040', 'properties': {'B_1984': 379.54935547707277, 'B_1985': 175.1008937837719, 'B_1986': 126.16541796612563, 'B_1987': 204.92193704562501, 'B_1988': 164.75858819285233, 'B_1989': 155.7085687173066, 'B_1990': 264.9866206921794, 'B_1991': 150.10114997215913, 'B_1992': 253.60466733415078, 'B_1993': 203.53236731140768, 'B_1994': 266.70357435083696, 'B_1995': 215.0291819486526, 'B_1996': 232.02530905991932, 'B_1997': 290.19870677147765, 'B_1998': 239.3311692385716, 'B_1999': 178.58875097704504, 'B_2000': 174.04495452656127, 'B_2001': 173.49336393651765, 'B_2002': 2

# 7. Exporting Data
**N.B.** data must be exported as both CSV and GeoJSON

In [ ]:
# Specify Google Drive Folder to export data to. 
exportFolder = 'Nat_Geo_Code'

# Define Export File Names
lastMined_outfile_name = '2021-03-09_lastMined_srHarmonizedMed_metricProcessed'
epaEcoregion_outfile_name = '2021-03-09_epaEcoregionSite_srHarmonizedMed_metricProcessed' 
customPolygon_outfile_name = '2021-03-09_customPolygon_srHarmonizedMed_metricProcessed'

In [ ]:
# Prepare Output Tasks
lastMined_csv = batch.Export.table.toDrive(processedLastMined_5yr,description=lastMined_outfile_name,folder=exportFolder,fileFormat='CSV')
lastMined_gjs = batch.Export.table.toDrive(processedLastMined_5yr,description=lastMined_outfile_name,folder=exportFolder,fileFormat='GeoJSON')

ecoRegion_csv = batch.Export.table.toDrive(fullyProcessed_EPA_ER_Sites,description=epaEcoregion_outfile_name,folder=exportFolder,fileFormat='CSV')
ecoRegion_gjs = batch.Export.table.toDrive(fullyProcessed_EPA_ER_Sites,description=epaEcoregion_outfile_name,folder=exportFolder,fileFormat='GeoJSON')

customPoly_csv = batch.Export.table.toDrive(fullyProcessedCustomPolygons,description=customPolygon_outfile_name,folder=exportFolder,fileFormat='CSV')
customPoly_gjs = batch.Export.table.toDrive(fullyProcessedCustomPolygons,description=customPolygon_outfile_name,folder=exportFolder,fileFormat='GeoJSON')

In [ ]:
# # Begin the Export Tasks
# exporting_lastMined_csv = batch.Task.start(lastMined_csv)
# exporting_lastMined_gjs = batch.Task.start(lastMined_gjs)

# exporting_ecoRegion_csv = batch.Task.start(ecoRegion_csv)
# exporting_ecoRegion_gjs = batch.Task.start(ecoRegion_gjs)

exporting_customPoly_csv = batch.Task.start(customPoly_csv)
exporting_customPoly_gjs = batch.Task.start(customPoly_gjs)

print("Export started, process(es) sent to cloud")

Export started, process(es) sent to cloud


# Fin - CT 2021-03-09